In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import betfairlightweight
import flumine
from flumine.resources import recorder
from flumine.storage import storageengine
import json
from datetime import datetime as dt
import datetime
import h2o

In [ ]:
from agent_bsp import Agent

In [ ]:
model = './models/bsp/XRT_1_AutoML_20200109_105830/XRT_1_AutoML_20200109_105830.zip'

In [ ]:
run_time = 5
mrkt = '1.167161599'
market_ids = [mrkt]

In [ ]:
end = dt.now() + datetime.timedelta(minutes=run_time)
end.strftime("%H:%M:%S")

In [ ]:
market_filter = {"marketIds": market_ids}
storage_engine = storageengine.Local('./data')

rcd =  recorder.MarketRecorder(storage_engine, market_filter=market_filter)

In [ ]:
with open('secrets/config.json', 'r') as fp:
    config = json.load(fp)

trading = betfairlightweight.APIClient(**config)
trading.login()

In [ ]:
current_orders = trading.betting.list_current_orders(market_ids=market_ids)

In [ ]:
current_orders.orders[0].size_matched * (current_orders.orders[0].average_price_matched - 1)

In [ ]:
current_orders.orders[0].__dict__

In [ ]:
bot = Agent(mrkt, trading, rcd, model)

In [ ]:
bot.init_stream()

In [ ]:
bot.flumine_object.status

In [ ]:
bot.poll_directory()
bot_df = bot.market_df()
bot_df[bot_df.sid==bot.sid].shape

In [ ]:
bot.assess_bsp(verbose=True)

In [ ]:
print(bot.current_direction)

In [ ]:
bot.trade_price

In [ ]:
bot.in_market

In [ ]:
while dt.now() < end:
    bot.poll_directory()
    if not bot.in_market:
        bot.assess_bsp(verbose=True)
    bot.manage_trade()
    
print('END END END')

In [ ]:
bot.cashout()

In [ ]:
trading.logout()